In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
car_reviews= pd.read_csv('/content/drive/My Drive/FatimaQurban_ML&DL_BytewiseFellowship2024/FatimaQurban_ML&DL_BWF_AnalyzeCarReviews/Car_Reviews.csv')


In [38]:
car_reviews.head(10)

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,Positive
1,The car is fine. It's a bit loud and not very ...,Negetive
2,My first foreign car. Love it I would buy anot...,Positive
3,I've come across numerous reviews praising the...,Negetive
4,I've been dreaming of owning an SUV for quite ...,Positive


In [39]:
!pip install transformers

In [40]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [41]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
! pip install nltk


In [43]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [44]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score
import nltk
from nltk.translate.bleu_score import sentence_bleu


# 1. Sentiment Classification and Evaluation
# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Extract reviews from the dataset
reviews = car_reviews['Review'].tolist()

# Perform sentiment classification
predicted_labels = sentiment_pipeline(reviews)

# Map the predicted labels to binary format (0 for Negative, 1 for Positive)
label_mapping = {'POSITIVE': 1, 'NEGATIVE': 0}
predictions = [label_mapping[pred['label'].upper()] for pred in predicted_labels]

# Clean and convert the original labels to binary format
car_reviews['Class'] = car_reviews['Class'].str.strip().str.upper()

# Filter out any invalid labels
valid_labels = car_reviews['Class'].isin(['POSITIVE', 'NEGATIVE'])
car_reviews = car_reviews[valid_labels]

# Convert valid labels to binary format
true_labels = car_reviews['Class'].map({'POSITIVE': 1, 'NEGATIVE': 0}).tolist()

# Ensure predictions list length matches true_labels
if len(predictions) != len(true_labels):
    predictions = predictions[:len(true_labels)]

# Calculate accuracy and F1 score
accuracy_result = accuracy_score(true_labels, predictions)
f1_result = f1_score(true_labels, predictions)

print("Accuracy:", accuracy_result)
print("F1 Score:", f1_result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Accuracy: 0.6666666666666666
F1 Score: 0.8


In [45]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Initialize translation pipeline (English to Spanish)
translation_pipeline = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

# Extract the first two sentences of the first review
first_review = reviews[0]
first_two_sentences = ' '.join(first_review.split('.')[:2])

# Translate the review
translated_review = translation_pipeline(first_two_sentences)[0]['translation_text']

# Load the reference translations from the file
reference_path = '/content/drive/My Drive/FatimaQurban_ML&DL_BytewiseFellowship2024/FatimaQurban_ML&DL_BWF_AnalyzeCarReviews/reference_translations.txt'
with open(reference_path, 'r') as f:
    references = [line.strip().split() for line in f]

# Tokenize the translated review (candidate)
candidate = translated_review.split()

# Calculate the BLEU score
bleu_score = sentence_bleu(references, candidate)

print(f"Translated Review: {translated_review}")
print(f"BLEU Score: {bleu_score}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated Review: Estoy muy satisfecho con mi Nissan NV SL 2014 Uso esta camioneta para mis entregas de negocios y uso personal
BLEU Score: 0.5793949630090935


In [46]:
from transformers import pipeline

#question = "What did the user initially liked about?"
question = "what are the problems in the car?"

# Context is the second review
context = """
The car is fine. It's a bit loud and not very powerful. On one hand compared to its peers the interior is well-built.
The transmission failed a few years ago and the dealer replaced it under warranty with no issues.
Now the transmission is failing again. It sounds like a truck and the issues are well-documented.
The dealer tells me it is normal refusing to do anything to resolve the issue.
After owning the car for 4 years there are many other vehicles I would purchase over this one.
Initially I really liked what the brand is about: ride quality reliability etc.
But I will not purchase another one. Despite these concerns I must say the level of comfort in the car has always been satisfactory but not worth the rest of issues found.
"""

# Initialize the QA pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Get the answer
answer = qa_pipeline(question=question, context=context)['answer']

print(answer)


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


It's a bit loud and not very powerful


In [47]:

# 4. Summarize the Last Review
last_review = reviews[-1]

# Initialize summarization pipeline
summarization_pipeline = pipeline("summarization")
summarized_text = summarization_pipeline(last_review,  min_length=50, do_sample=False)[0]['summary_text']

print( summarized_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 The Nissan Rogue provides the desired SUV experience without burdening me with an exorbitant payment . Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more . The engine delivers strong performance, and the ride is really smooth .
